In [ ]:
!which python
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from qtrain.utils.defaults import *


import monai.losses as L
import monai.metrics as M
import segmentation_models_pytorch as smp

from monai.transforms import CropForeground
from torch.utils.data import Dataset, DataLoader

from qtrain.dataset.infarct import InfarctDataset3D
from qtrain.utils import apply_torch_transform_3d, apply_torch_transform

In [ ]:
with open("3dunet_template.json", "r") as f:
    args = munch.munchify(json.load(f))

In [ ]:
args.gpu = [1]
args.n_slices = 8
args.img_size = 224
args.max_epoch = 300
args.task = ["segmentation"]
args.monitor = "valid_loss"

args.batch_size = 8
args.val_batch_size = 8

args.only_annotated = False
args.augmentation =  True
args.extra_augs = False

args.windowing = "gray-white-matter"

args.loss_contrib = [2,3]
args.losses = [L.FocalLoss(), L.DiceCELoss()]

args.metrics = [M.DiceMetric(include_background=True, reduction="mean"),
                M.HausdorffDistanceMetric(include_background=True, reduction="mean")]

args.optimizer = torch.optim.Adam
args.optimizer_params = munch.munchify({
    'lr': 5e-3,
    'weight_decay' : 4e-5
})

args.scheduler = torch.optim.lr_scheduler.MultiStepLR
args.scheduler_params = munch.munchify({
    'milestones': [25, 75, 150, 230], 
    'gamma' : 0.1
})

args.model_params = munch.munchify({
    "spatial_dims": 3,
    "in_channels": 3, 
    "out_channels": 1, 
    "channels": (16, 32, 64, 128), 
    "strides": (2, 2, 2), 
    "kernel_size": 3, 
    "up_kernel_size": 3, 
    "num_res_units": 0, 
    "act": 'PRELU', 
    "norm": 'INSTANCE', 
    "dropout": 0.0, 
    "bias": True, 
    "adn_ordering": 'NDA'
})

In [ ]:
args.fast_batch_size = 20
args.fast_dev_run = False

with open("3dunet_22march.yaml", "w") as f:
    yaml.dump(args, f)

In [ ]:
with open("3dunet_22march.yaml", "r") as f:
    args_yaml = munch.munchify(yaml.load(f, Loader=yaml.Loader))

In [ ]:
# args.optimizer_name(net.parameters(), **args.optimizer)

In [ ]:
dataset = InfarctDataset3D(args_yaml)

In [ ]:
loader = DataLoader(
            dataset,
            batch_size=4,
            num_workers=5,
            drop_last=True,
            pin_memory=True,
            shuffle= True
        )

In [ ]:
args_yaml.model_params

In [ ]:
from monai.networks.nets import UNet
device = torch.device("cuda:0")
net = UNet(**args_yaml.model_params).double().to(device)

In [ ]:
a, b = next(iter(loader))
a.shape, b.shape

In [ ]:
b_ = net(a.double().to(device))
b_.shape